<a href="https://colab.research.google.com/github/higor-gomes93/employee_attrition/blob/main/Employee_Attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Completo - Saídas de Colaboradores**

#### **Problema**
Iremos realizar uma análise completa de dados envolvendo o problema de predição de saídas de colaboradores em uma empresa. Os dados utilizados podem ser encontrados [aqui](https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset). A anáise será dividida em quatro etapas:
1. Análise básica: conexão com a base de dados, limpeza completa do dataset e análise exploratória gráfica
2. Análise intermediária: correlações, testes de hipóteses e análise de sobrevivência
3. Análise avançada: modelos preditivos, feature importance, oversampling e escolha de um modelo
4. Análise expert: feature selection, feature engineering, grid search, random search e avaliação das métricas de sucesso

#### **Objetivos**
Identificar quais são as principais alavancas que estão levando as pessoas deixarem a empresa, validar algumas hipóteses e desenvolver um modelo preditivo para essas saídas.




---



## Parte 1 - Análise Básica

Instalação e importação das bibliotecas, conexão com os dados, limpeza do dataset e análise exploratória completa.

### 1) Instalação e Importação das Bibliotecas

Nesta etapa iremos importar as bibliotecas e pacotes necessários para fazer todo o projeto. Conforme visto no módulo de introdução à Python, existem diversas bibliotecas prontas para facilitar o trabalho de análise de dados, e iremos utilizar algumas delas aqui.

#### 1.1) Instalação das bibliotecas necessárias

In [ ]:
# Instalação das bibliotecas
!pip install -q lifelines
!pip install chart-studio

     |████████████████████████████████| 349 kB 7.2 MB/s 
     |████████████████████████████████| 76 kB 2.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.0 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.3.0 which is incompatible.
spacy 3.4.1 requires typing-extensions<4.2.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.3.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 1.9 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=5ead1bb5dbcfe24db70f6cb126c1558f6a8d3dec4612242e9450af4e07517c6b
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built

#### 1.2) Importação das bibliotecas necessárias

In [ ]:
# Bibliotecas básicas
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None  # Removendo warnings
import warnings
warnings.filterwarnings('ignore') # Removendo warnings
import numpy as np
import datetime as dt
import statistics as st
import itertools
from scipy import stats
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot
import random as rd

# Bibliotecas de visualização
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as po
from plotly.subplots import make_subplots
import plotly.tools as tls
import plotly.figure_factory as ff
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as cstls

# Importação das bibliotecas de predição
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.feature_selection import SelectKBest, RFE, SelectFromModel
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, r2_score, mean_squared_error, precision_score, recall_score, max_error, roc_auc_score, roc_curve, precision_recall_curve
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import export_graphviz, DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import RandomOverSampler

# Bibliotecas adicionais
from lifelines import KaplanMeierFitter
from dateutil.relativedelta import relativedelta
import graphviz
from sklearn.tree import export_graphviz
from io import StringIO
from IPython.display import Image  
import pydotplus
import math

### 2) Conexão com a Base de Dados

Iremos importar diretamente para nosso código a base de dados dos colaboradores. Ela está localizada em uma pasta no Google Drive, e é possível fazer essa conexão diretamente.

In [ ]:
# Pegando o arquivo csv no diretório
path = 'https://raw.githubusercontent.com/higor-gomes93/employee_attrition/main/employee_attrition.csv'
dataset_inicial = pd.read_csv(path)

In [ ]:
# Conferindo a conexão
dataset_inicial


### 3) Observação dos Dados

Nessa etapa iremos analisar previamente os dados, observando quais atributos temos disponíveis, o total de observações, algumas medidas estatísticas, tratamentos a serem feitos, o formato e o tipo dos dados, entre outras.

#### 3.1 Observações iniciais

In [ ]:
# Observando o formato do dataset inicial
dataset_inicial.head(3)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0


In [ ]:
# Observando o total de linhas e colunas
dataset_inicial.shape

(1470, 35)

In [ ]:
# Observando o nome das colunas
dataset_inicial.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [ ]:
# Observando algumas medidas estatísticas do dataset inicial
dataset_inicial.describe()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,1.0,1024.865306,2.721769,65.891156,2.729932,2.063946,2.728571,6502.931293,14313.103401,2.693197,15.209524,3.153741,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,1.024165,0.0,602.024335,1.093082,20.329428,0.711561,1.106940,1.102846,4707.956783,7117.786044,2.498009,3.659938,0.360824,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,1.000000,1009.000000,2094.000000,0.000000,11.000000,3.000000,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,1.000000,2.000000,2911.000000,8047.000000,1.000000,12.000000,3.000000,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,66.000000,3.000000,2.000000,3.000000,4919.000000,14235.500000,2.000000,14.000000,3.000000,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1.0,1555.750000,4.000000,83.750000,3.000000,3.000000,4.000000,8379.000000,20461.500000,4.000000,18.000000,3.000000,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,5.000000,4.000000,19999.000000,26999.000000,9.000000,25.000000,4.000000,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [ ]:
# Observando todas as medidas estatísticas do dataset inicial
dataset_inicial.describe(include = 'all')

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470,1470,1470.000000,1470,1470.000000,1470.000000,1470,1470.0,1470.000000,1470.000000,1470,1470.000000,1470.000000,1470.000000,1470,1470.000000,1470,1470.000000,1470.000000,1470.000000,1470,1470,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
unique,NaN,2,3,NaN,3,NaN,NaN,6,NaN,NaN,NaN,2,NaN,NaN,NaN,9,NaN,3,NaN,NaN,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,No,Travel_Rarely,NaN,Research & Development,NaN,NaN,Life Sciences,NaN,NaN,NaN,Male,NaN,NaN,NaN,Sales Executive,NaN,Married,NaN,NaN,NaN,Y,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1233,1043,NaN,961,NaN,NaN,606,NaN,NaN,NaN,882,NaN,NaN,NaN,326,NaN,673,NaN,NaN,NaN,1470,1054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,36.923810,NaN,NaN,802.485714,NaN,9.192517,2.912925,NaN,1.0,1024.865306,2.721769,NaN,65.891156,2.729932,2.063946,NaN,2.728571,NaN,6502.931293,14313.103401,2.693197,NaN,NaN,15.209524,3.153741,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,NaN,NaN,403.509100,NaN,8.106864,1.024165,NaN,0.0,602.024335,1.093082,NaN,20.329428,0.711561,1.106940,NaN,1.102846,NaN,4707.956783,7117.786044,2.498009,NaN,NaN,3.659938,0.360824,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,NaN,NaN,102.000000,NaN,1.000000,1.000000,NaN,1.0,1.000000,1.000000,NaN,30.000000,1.000000,1.000000,NaN,1.000000,NaN,1009.000000,2094.000000,0.000000,NaN,NaN,11.000000,3.000000,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,NaN,NaN,465.000000,NaN,2.000000,2.000000,NaN,1.0,491.250000,2.000000,NaN,48.000000,2.000000,1.000000,NaN,2.000000,NaN,2911.000000,8047.000000,1.000000,NaN,NaN,12.000000,3.000000,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,NaN,NaN,802.000000,NaN,7.000000,3.000000,NaN,1.0,1020.500000,3.000000,NaN,66.000000,3.000000,2.000000,NaN,3.000000,NaN,4919.000000,14235.500000,2.000000,NaN,NaN,14.000000,3.000000,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,NaN,NaN,1157.000000,NaN,14.000000,4.000000,NaN,1.0,1555.750000,4.000000,NaN,83.750000,3.000000,3.000000,NaN,4.000000,NaN,8379.000000,20461.500000,4.000000,NaN,NaN,18.000000,3.000000,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000


In [ ]:
# Algumas informações adicionais
dataset_inicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

#### 3.2) Observando valores nulos

In [ ]:
# Identificando valores nulos
dataset_inicial.isnull().any()

Age                         False
Attrition                   False
BusinessTravel              False
DailyRate                   False
Department                  False
DistanceFromHome            False
Education                   False
EducationField              False
EmployeeCount               False
EmployeeNumber              False
EnvironmentSatisfaction     False
Gender                      False
HourlyRate                  False
JobInvolvement              False
JobLevel                    False
JobRole                     False
JobSatisfaction             False
MaritalStatus               False
MonthlyIncome               False
MonthlyRate                 False
NumCompaniesWorked          False
Over18                      False
OverTime                    False
PercentSalaryHike           False
PerformanceRating           False
RelationshipSatisfaction    False
StandardHours               False
StockOptionLevel            False
TotalWorkingYears           False
TrainingTimesL

#### 3.3) Observando outliers

In [ ]:
# Criando dicionários para adicioinar os valores
dict_outliers_zscore = {}
dict_outliers_iqr = {}
dict_outliers_percentile = {}

In [ ]:
# Definindo uma função para coleta de outliers por meio do método Z-score com maior robustez
def z_r_score_outlier(data):
    outliers = []
    med = np.median(data)
    ma = stats.median_absolute_deviation(data)
    for i in data: 
        z = (0.6745*(i-med))/ (np.median(ma))
        if np.abs(z) > 3: 
            outliers.append(i)
    return outliers

# Definindo uma função para coleta de outliers por meio do método IQR
def iqr_outliers(data):
    outliers = []
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3-q1
    Lower_tail = q1 - 2.5 * iqr
    Upper_tail = q3 + 2.5 * iqr
    for i in data:
        if i > Upper_tail or i < Lower_tail:
            outliers.append(i)
    return outliers

# Definindo uma função para coleta de outliers por meio da análise do percentil
def winsorization_outliers(data):
    outliers = []
    q1 = np.percentile(data, 1)
    q3 = np.percentile(data, 99)
    for i in data:
        if i > q3 or i < q1:
            outliers.append(i)
    return outliers

In [ ]:
# Coletando os outliers
for i in dataset_inicial.columns:
    if dataset_inicial[i].dtype == 'int64':
        outliers_zscore = z_r_score_outlier(dataset_inicial[i])
        if len(outliers_zscore) > 0:
            dict_outliers_zscore[i] = outliers_zscore
        outliers_iqr = iqr_outliers(dataset_inicial[i])
        if len(outliers_iqr) > 0:
            dict_outliers_iqr[i] = outliers_iqr
        outliers_percentile = winsorization_outliers(dataset_inicial[i])
        if len(outliers_percentile) > 0:
            dict_outliers_percentile[i] = outliers_percentile

### 4) Limpeza e Tratamento dos Dados

Etapa de remoção de outliers, padronização de categorias, tratamento de valores faltantes e outras transformações necessárias.

#### 4.1) Removendo colunas sem valor semântico

In [ ]:
# Colunas sem importância para as análises
dataset_inicial = dataset_inicial.drop(columns = ['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'])

#### 4.2) Valores faltantes

In [ ]:
# Conforme indicado anteriormente, não existem valores nulos na base de dados
print('Não existem valores nulos.') if any(list(dataset_inicial.isnull().any())) == False else print('Existem valores nulos.')

Não existem valores nulos.


#### 4.3) Outliers e valores fora do padrão

In [ ]:
# Iremos nos basear na técnica que analise o percentil dos dados
# Coletando os outliers
dict_outliers_percentile = {}
for i in dataset_inicial.columns:
    if dataset_inicial[i].dtype == 'int64':
        outliers_percentile = winsorization_outliers(dataset_inicial[i])
        if len(outliers_percentile) > 0:
            dict_outliers_percentile[i] = outliers_percentile

# Observando os outliers
for i in dict_outliers_percentile.items():
    print(i)

('Age', [59, 59, 59, 59, 59, 59, 18, 18, 60, 60, 18, 60, 18, 59, 59, 18, 60, 59, 59, 18, 18, 60, 18])
('DailyRate', [103, 1488, 111, 1496, 111, 106, 1490, 1490, 1499, 1495, 102, 109, 1492, 111, 116, 107, 1498, 1495, 1490, 1496, 115, 104, 1495, 1490, 116, 105])
('MonthlyIncome', [1232, 19926, 1102, 19999, 1200, 1009, 1281, 19859, 1051, 19973, 19845, 1052, 19627, 19943, 19740, 1223, 1118, 19847, 19717, 19701, 1359, 1261, 1274, 19658, 19833, 19665, 1081, 1091, 19636, 1129])
('MonthlyRate', [2094, 26959, 26897, 26820, 2302, 2137, 26767, 26707, 26914, 2227, 2288, 2112, 2125, 26894, 26999, 2104, 2243, 26968, 2253, 26933, 2323, 2261, 2097, 26997, 26841, 2125, 2122, 26862, 26849, 26956])
('TotalWorkingYears', [0, 37, 38, 40, 0, 36, 37, 0, 0, 36, 37, 0, 40, 0, 0, 0, 36, 36, 0, 36, 0, 36, 37, 0])
('YearsAtCompany', [37, 40, 33, 33, 36, 32, 34, 32, 33, 33, 32, 33, 36])
('YearsInCurrentRole', [16, 18, 18, 17, 16, 16, 16, 16, 16, 17, 17, 17, 16])
('YearsSinceLastPromotion', [15, 15, 15, 15, 15, 15,

In [ ]:
'''
Observando os valores de outliers obtidos, será tomada a decisão de não eliminar nenhum 
registro (ou substituí-lo), pois eles têm um significado no contexto de predição de saídas.
'''

'\nObservando os valores de outliers obtidos, será tomada a decisão de não eliminar nenhum \nregistro (ou substituí-lo), pois eles têm um significado no contexto de predição de saídas.\n'

#### 4.4) Variáveis categóricas fora do padrão

In [ ]:
# Selecionando as variáveis categóricas
variaveis_categoricas = dataset_inicial.select_dtypes(exclude = ["int64"])

# Observando se existem valores despadronizados
for i in variaveis_categoricas:
    aux = dataset_inicial.groupby([i]).count().reset_index().iloc[:, 0:2]
    aux.columns = [i, 'Contagem']
    display(aux)
    print("\n\n")

,Attrition,Contagem
0,No,1233
1,Yes,237


,BusinessTravel,Contagem
0,Non-Travel,150
1,Travel_Frequently,277
2,Travel_Rarely,1043


,Department,Contagem
0,Human Resources,63
1,Research & Development,961
2,Sales,446


,EducationField,Contagem
0,Human Resources,27
1,Life Sciences,606
2,Marketing,159
3,Medical,464
4,Other,82
5,Technical Degree,132


,Gender,Contagem
0,Female,588
1,Male,882


,JobRole,Contagem
0,Healthcare Representative,131
1,Human Resources,52
2,Laboratory Technician,259
3,Manager,102
4,Manufacturing Director,145
5,Research Director,80
6,Research Scientist,292
7,Sales Executive,326
8,Sales Representative,83


,MaritalStatus,Contagem
0,Divorced,327
1,Married,673
2,Single,470


,OverTime,Contagem
0,No,1054
1,Yes,416


#### 4.5) Criando uma cópia

In [ ]:
dataset = dataset_inicial.copy()

### 5) Análise Exploratória do Dados

Iremos observar o dataset de forma gráfica, a fim de identificar alguns pontos que possam servir para análises futuras, além de entender as características e comportamentos da base.

#### 5.1) Distribuição da variável alvo

In [ ]:
# Função de criação de dois datasets auxiliares
def dataset_ativos_inativos(dataset = "Dataset a ser utilizado") -> object:
    dataset_inativos = dataset[(dataset['Attrition'] == 'Yes')]
    dataset_ativos = dataset[(dataset['Attrition'] == 'No')]
    return dataset_inativos, dataset_ativos

In [ ]:
# Importando os datasets
dataset_inativos = dataset_ativos_inativos(dataset)[0]
dataset_ativos = dataset_ativos_inativos(dataset)[1]

# Porcentagem
percent_lista = [f'{round(len(dataset_inativos)/len(dataset), 3)*10*10}%', f'{round(len(dataset_ativos)/len(dataset), 3)*10*10}%']

# Criando um visual de barras
trace_1 = go.Bar(x = (len(dataset_inativos), len(dataset_ativos)), 
                 y = ['Inativos', 'Ativos'], 
                 orientation = 'h', 
                 opacity = 0.8,
                 text = percent_lista,
                 marker = dict(color = ['lightsalmon', 'lightgreen'], line = dict(color = '#000000', width = 1.5)))

# Plotando os gráficos
fig_1 = go.Figure(trace_1)
fig_1.update_layout(width = 1000, height = 500, title = "Contagem da variável alvo", plot_bgcolor = "white", title_x = 0.5)
fig_1.update_yaxes(showline = True, linewidth = 1, linecolor = 'black')
fig_1.update_xaxes(showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE')
fig_1.show()

#### 5.2) Variáveis numéricas - distribuições

In [ ]:
# Selecionando as variáveis numéricas de interesse (sem as categóricas)
colunas_dist_plot_num = dataset.nunique()[dataset.nunique() > 10].keys().tolist()
colunas_dist_plot_num = [x for x in colunas_dist_plot_num]

In [ ]:
# Definindo a função de plotagem
def dist_plot(variaveis = "Lista de features de interesse", dataset = dataset, width = 1500, height = 2100) -> object:
    # Definindo um limite superior para a plotagem
    limite_superior = int(len(variaveis)/2) if len(variaveis)/2 == len(variaveis)//2 else len(variaveis)//2 + 1

    if limite_superior > 1:
        cols = 2
    else:
        cols = 1

    # Datasets auxiliares
    dataset_inativos = dataset_ativos_inativos(dataset)[0]
    dataset_ativos = dataset_ativos_inativos(dataset)[1]

    # Plotando todos os gráficos
    fig = make_subplots(rows = limite_superior, cols = cols, subplot_titles = variaveis)
    contador = 0

    for i in range(1, limite_superior+1):
        for j in range(1, cols+1):
            try:
                # Legenda
                legenda = True if contador == 0 else False

                # Criando os datasets auxiliares
                aux_1 = dataset_inativos[variaveis[contador]]
                aux_2 = dataset_ativos[variaveis[contador]]
                
                # Criando os atributos do gráfico
                data = [aux_1, aux_2]
                labels = ['Inativos', 'Ativos']
                colors = ['#FFA07A', '#90EE90']

                # Obtendo os dados 
                fig1 = ff.create_distplot(hist_data = data, group_labels = labels, colors = colors, show_rug = False, bin_size = False)

                # Criando o gráfico
                fig.add_trace(go.Histogram(fig1['data'][0], 
                                        marker_color = colors[0], 
                                        showlegend = legenda, 
                                        opacity = 0.6), 
                            row = i, col = j)
                fig.add_trace(go.Histogram(fig1['data'][1], 
                                        marker_color = colors[1], 
                                        showlegend = legenda, 
                                        opacity = 0.6), 
                            row = i, col = j)
                fig.add_trace(go.Scatter(fig1['data'][2], 
                                        line = dict(color = colors[0], width = 1.5)), 
                            row = i, col = j)
                fig.add_trace(go.Scatter(fig1['data'][3], 
                                        line = dict(color = colors[1], width = 1.5)),
                            row = i, col = j)
                contador += 1
            except:
                pass

    fig.update_layout(width = width, height = height, title = "Distribuição das variáveis numéricas de interesse", plot_bgcolor = "white", title_x = 0.5)
    fig.update_yaxes(showline = True, linewidth = 1, linecolor = 'black', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE', rangemode = 'tozero')
    fig.update_xaxes(showline = True, linewidth = 1, linecolor = 'black', rangemode = 'nonnegative')
    fig.show()
    

# Realizando a plotagem
dist_plot(colunas_dist_plot_num)

#### 5.3) Variáveis numéricas - barras

In [ ]:
# Selecionando as variáveis numéricas de interesse (sem as categóricas)
colunas_bar_plot_num = dataset.nunique()[(dataset.nunique() < 50) & (dataset.nunique() > 10)].keys().tolist()
colunas_bar_plot_num = [x for x in colunas_bar_plot_num]

In [ ]:
# Definindo a função de plotagem
def bar_plot_num(variaveis = "Lista de features de interesse", dataset = dataset, width = 1500, height = 2100) -> object:
    # Definindo um limite superior para a plotagem
    limite_superior = int(len(variaveis)/2) if len(variaveis)/2 == len(variaveis)//2 else len(variaveis)//2 + 1
    
    if limite_superior > 1:
        cols = 2
        specs = [[{'secondary_y': True}, {'secondary_y': True}]]
    else:
        cols = 1
        specs = [[{'secondary_y': True}]]

    # Datasets auxiliares
    dataset_inativos = dataset_ativos_inativos(dataset)[0]
    dataset_ativos = dataset_ativos_inativos(dataset)[1]

    # Plotando todos os gráficos
    fig = make_subplots(rows = limite_superior, cols = cols, subplot_titles = variaveis, specs = specs*limite_superior)
    contador = 0

    for i in range(1, limite_superior+1):
        for j in range(1, cols+1):
            try:
                # Legenda
                legenda = True if contador == 0 else False

                # Criando os datasets auxiliares
                aux_1 = dataset_inativos
                aux_2 = dataset_ativos
                aux_3 = pd.DataFrame(pd.crosstab(dataset[variaveis[contador]], dataset['Attrition']), )
                aux_3['% de Saídas'] = aux_3['Yes'] / (aux_3['Yes'] + aux_3['No']) * 100
                aux_3.columns = ['No', 'Yes', '% de Saídas']
                aux_3.index.name = None
                
                # Criando os atributos do gráfico
                data = [aux_1, aux_2]
                labels = ['Inativos', 'Ativos']
                colors = ['#FFA07A', '#90EE90']

                # Criando o gráfico
                fig.add_trace(go.Bar(x = aux_1[variaveis[contador]].value_counts().keys().tolist(), 
                                        y = aux_1[variaveis[contador]].value_counts().values.tolist(), 
                                        name = 'Inativos', 
                                        opacity = 0.8,
                                        showlegend = legenda,
                                        marker = dict(color = 'LightSalmon', line = dict(color = '#000000', width = 1))), 
                                secondary_y = False,
                                row = i, col = j)
                fig.add_trace(go.Bar(x = aux_2[variaveis[contador]].value_counts().keys().tolist(), 
                                        y = aux_2[variaveis[contador]].value_counts().values.tolist(), 
                                        name = 'Ativos', 
                                        opacity = 0.8,
                                        showlegend = legenda, 
                                        marker = dict(color = 'LightGreen', line = dict(color = '#000000', width = 1))), 
                                secondary_y = False,
                                row = i, col = j)
                fig.add_trace(go.Scatter(x = aux_3.index, 
                                            y = aux_3['% de Saídas'], 
                                            name = '% de Saídas', 
                                            opacity = 0.6, 
                                            showlegend = legenda,
                                            marker = dict(color = 'black', line = dict(color = '#000000', width = 0.5))), 
                                secondary_y = True,
                                row = i, col = j)
                contador += 1
            except:
                pass

    fig.update_layout(width = width, height = height, title = "Barras das variáveis numéricas de interesse", plot_bgcolor = "white", title_x = 0.5)
    fig.update_yaxes(showline = False, linewidth = 1, linecolor = 'black', rangemode = 'tozero', secondary_y = False, title = "Contagem", showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE')
    fig.update_yaxes(showline = False, linewidth = 1, linecolor = 'black', rangemode = 'tozero', secondary_y = True, title = "% de Saídas", range = [-0, 100])
    fig.update_xaxes(showline = True, linewidth = 1, linecolor = 'black', rangemode = 'nonnegative')
    fig.show()

# Realizando a plotagem
bar_plot_num(colunas_bar_plot_num)

#### 5.4) Variáveis categóricas - barras

In [ ]:
# Selecionando as variáveis numéricas de interesse (categóricas)
colunas_bar_plot_cat = dataset.nunique()[dataset.nunique() < 10].keys().tolist()
colunas_bar_plot_cat = [x for x in colunas_bar_plot_cat]
colunas_bar_plot_cat.remove('Attrition')

In [ ]:
# Definindo a função de plotagem
def bar_plot_cat(variaveis = "Lista de features de interesse", dataset = dataset, width = 1500, height = 2500) -> object:
    # Definindo um limite superior para a plotagem
    limite_superior = int(len(variaveis)/2) if len(variaveis)/2 == len(variaveis)//2 else len(variaveis)//2 + 1
    
    if limite_superior > 1:
        cols = 2
        specs = [[{'secondary_y': True}, {'secondary_y': True}]]
    else:
        cols = 1
        specs = [[{'secondary_y': True}]]

    # Datasets auxiliares
    dataset_inativos = dataset_ativos_inativos(dataset)[0]
    dataset_ativos = dataset_ativos_inativos(dataset)[1]

    # Plotando todos os gráficos
    fig = make_subplots(rows = limite_superior, cols = cols, subplot_titles = variaveis, specs = specs*limite_superior)
    contador = 0

    for i in range(1, limite_superior+1):
        for j in range(1, cols+1):
            try:
                # Legenda
                legenda = True if contador == 0 else False

                # Criando os datasets auxiliares
                aux_1 = dataset_inativos
                aux_2 = dataset_ativos
                aux_3 = pd.DataFrame(pd.crosstab(dataset[variaveis[contador]], dataset['Attrition']), )
                aux_3['% de Saídas'] = aux_3['Yes'] / (aux_3['Yes'] + aux_3['No']) * 100
                aux_3.columns = ['No', 'Yes', '% de Saídas']
                aux_3.index.name = None
                if isinstance(aux_3.index[0], str):
                    aux_3 = aux_3.sort_values(by = 'Yes', ascending = False)
                
                # Criando os atributos do gráfico
                data = [aux_1, aux_2]
                labels = ['Inativos', 'Ativos']
                colors = ['#FFA07A', '#90EE90']

                # Criando o gráfico
                fig.add_trace(go.Bar(x = aux_1[variaveis[contador]].value_counts().keys().tolist(), 
                                        y = aux_1[variaveis[contador]].value_counts().values.tolist(), 
                                        name = 'Inativos', 
                                        opacity = 0.8,
                                        showlegend = legenda,
                                        marker = dict(color = 'LightSalmon', line = dict(color = '#000000', width = 1))), 
                                secondary_y = False,
                                row = i, col = j)
                fig.add_trace(go.Bar(x = aux_2[variaveis[contador]].value_counts().keys().tolist(), 
                                        y = aux_2[variaveis[contador]].value_counts().values.tolist(), 
                                        name = 'Ativos', 
                                        opacity = 0.8,
                                        showlegend = legenda, 
                                        marker = dict(color = 'LightGreen', line = dict(color = '#000000', width = 1))), 
                                secondary_y = False,
                                row = i, col = j)
                fig.add_trace(go.Scatter(x = aux_3.index, 
                                            y = aux_3['% de Saídas'], 
                                            name = '% de Saídas', 
                                            opacity = 0.6, 
                                            showlegend = legenda,
                                            marker = dict(color = 'black', line = dict(color = '#000000', width = 0.5))), 
                                secondary_y = True,
                                row = i, col = j)
                contador += 1
            except:
                pass

    fig.update_layout(width = width, height = height, title = "Barras das variáveis categóricas de interesse", plot_bgcolor = "white", title_x = 0.5)
    fig.update_yaxes(showline = False, linewidth = 1, linecolor = 'black', rangemode = 'tozero', secondary_y = False, title = "Contagem", showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE')
    fig.update_yaxes(showline = False, linewidth = 1, linecolor = 'black', rangemode = 'tozero', secondary_y = True, title = "% de Saídas", range = [-0, 100])
    fig.update_xaxes(showline = True, linewidth = 1, linecolor = 'black', rangemode = 'nonnegative')
    fig.show()
    #py.plot(fig, filename=str(variaveis[0])+'barras2', auto_open = False)

# Realizando a plotagem
bar_plot_cat(colunas_bar_plot_cat)

In [ ]:
'''
for x in colunas_bar_plot_cat:
    bar_plot_cat([x], width = 1200, height = 628)
'''

'\nfor x in colunas_bar_plot_cat:\n    bar_plot_cat([x], width = 1200, height = 628)\n'

#### 5.5) Matrix de scatter plot

In [ ]:
# Selecionando as variáveis numéricas
colunas_scatter = colunas_bar_plot_cat.copy()
colunas_scatter.append("Attrition")

# Criando um dataset numérico
dataset_numerico_aux = dataset.drop(colunas_scatter, axis = 1)

# Criando um dataset apenas com a variável attrition
dataset_attrition = dataset[['Attrition']]

# Unindo os datasets (para alterar a ordem)
dataset_numerico = pd.concat([dataset_attrition, dataset_numerico_aux], axis = 1)

In [ ]:
# Definindo a função de plotagem
def scatter_matrix(dataset = "Dataset", width = 2500, height = 3000) -> object:
    # Garantindo o tipo
    dataset_aux = dataset.copy()
    dataset_aux['Attrition'] = dataset_aux['Attrition'].apply(lambda x: "Inativo" if x == 1 else  x)
    dataset_aux['Attrition'] = dataset_aux['Attrition'].apply(lambda x: "Ativo" if x == 0 else x)
    # Criando o visual para todas as variáveis numéricas
    fig = px.scatter_matrix(dataset_aux, 
                            dimensions = list(dataset_aux.columns)[1:-1], 
                            color = list(dataset_aux.columns)[0],
                            color_discrete_sequence = ['#FFA07A', '#90EE90'],
                            labels = ['Inativos', 'Ativos']
                        )

    # Atualizando o layout geral
    fig.update_layout(width = width, height = height, title = "Scatterplot Matrix com todas as variáveis numéricas", title_x = 0.5)
    fig.update_traces(opacity = 0.75)
    fig.show()
    #py.plot(fig, filename="Matriz de Scatter Plot", auto_open = False)

# Realizando a plotagem
scatter_matrix(dataset_numerico)

In [ ]:
# Selecionando as variáveis numéricas com maior variabilidade
colunas_cont_num = dataset.nunique()[dataset.nunique() > 30].keys().tolist()
colunas_cont_num = [x for x in colunas_cont_num]

# Criando um dataset apenas com as variáveis de interesse
colunas_scatter_2 = colunas_cont_num.copy()

# Criando um dataset numérico
dataset_numerico_2_aux = dataset[colunas_scatter_2]

# Unindo os datasets (para alterar a ordem)
dataset_numerico_2 = pd.concat([dataset_attrition, dataset_numerico_2_aux], axis = 1)

In [ ]:
# Realizando a plotagem
scatter_matrix(dataset_numerico_2, width = 1500, height = 1500)

## Parte 2 - Análise Intermediária

Correlaçãoes, testes de hipóteses e análise de sobrevivência.

### 6) Tratamento Adicional ao Dataset

Iremos aqui realizar o processo de transformação das variáveis categóricas e também a normalização dos dados.

#### 6.1) Codificação das features com Label Encoder

In [ ]:
# Criando uma cópia do dataset original para mantê-lo
dataset_label_encoder = dataset.copy()

In [ ]:
# Selecionando as features categóricas
features_categoricas = dataset_label_encoder[colunas_bar_plot_cat].select_dtypes(exclude = ["int64"]).keys().tolist()

# Criando um objeto codificador
le = LabelEncoder()

# Codificando as colunas
for i in features_categoricas:
    dataset_label_encoder[i] = le.fit_transform(dataset_label_encoder[i])

In [ ]:
# Alterando a ordem da variável alvo
dataset_label_encoder = dataset_label_encoder.drop(['Attrition'], axis = 1)
dataset_label_encoder = pd.concat([dataset_attrition, dataset_label_encoder], axis = 1)

# Codificando a variável alvo
dataset_label_encoder['Attrition'] = dataset_label_encoder['Attrition'].apply(lambda x: 1 if x == "Yes" else 0)

# Conferindo o tipo das colunas
set(list(dataset_label_encoder.dtypes))

{dtype('int64')}

#### 6.2) Utilizando a técnica de variáveis dummy para codificar

In [ ]:
# Criando uma cópia para facilitar o manueio
dataset_dummy = dataset.copy()

In [ ]:
# Variável alvo
target_col = ["Attrition"]

# Features categóricas
cat_cols   = dataset_dummy.nunique()[dataset_dummy.nunique() < 10].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]

# Features numéricas
num_cols   = [x for x in dataset_dummy.columns if x not in cat_cols + target_col]

# Features binárias
bin_cols   = dataset_dummy.nunique()[dataset_dummy.nunique() == 2].keys().tolist()

# Colunas com mais de dois valores
multi_cols = [i for i in cat_cols if i not in bin_cols]

In [ ]:
# Utilizando Label Encoding apenas para as variáveis binárias (0 e 1)
le = LabelEncoder()
for i in bin_cols :
    dataset_dummy[i] = le.fit_transform(dataset_dummy[i])
    
# Utilizando a técnica de variáveis dummy para features com mais de dois valores
dataset_dummy = pd.get_dummies(data = dataset_dummy, columns = multi_cols)

#### 6.3) Normalização das features

In [ ]:
# Normalizando entre 0 e 1 o dataset_label_encoder
min_max_scaler = MinMaxScaler()
normalizadas = min_max_scaler.fit_transform(dataset_label_encoder)
dataset_label_encoder_norm = pd.DataFrame(normalizadas, columns = dataset_label_encoder.columns)

# Deixando o dataset numa escala padronizada
std = StandardScaler()
dataset_scaled = std.fit_transform(dataset_label_encoder)
dataset_label_encoder_scaled = pd.DataFrame(dataset_scaled, columns = dataset_label_encoder.columns)
dataset_label_encoder_scaled['Attrition'] = dataset_label_encoder_scaled['Attrition'].apply(lambda x: 1 if x == max(dataset_label_encoder_scaled['Attrition']) else 0)

In [ ]:
# Deixando o dataset_dummy numa escala padronizada
min_max_scaler = None
min_max_scaler = MinMaxScaler()
dummy_norm = min_max_scaler.fit_transform(dataset_dummy[num_cols])
dataset_dummy_norm_aux = pd.DataFrame(dummy_norm, columns = num_cols)

# Unindo ao dataset_dummy numérico
dataset_dummy_original = dataset_dummy.copy()
dataset_dummy_norm = dataset_dummy.drop(columns = num_cols, axis = 1)
dataset_dummy_norm = dataset_dummy_norm.merge(dataset_dummy_norm_aux, left_index = True, right_index = True, how = "left")

### 7) Correlação

Etapa dedicada à plotagem da matriz de correlação, assim como a análise de variáveis colineares.

#### 7.1) Matrizes de correlação

In [ ]:
# Definindo a função de plotagem
def matriz_correlacao(dataset = "Dataset", width = 800, height = 800) -> object:
    # Obtendo as correlações
    correl_data = dataset.corr()

    # Obtendo as colunas
    colunas_correl = correl_data.columns.tolist()

    # Convertendo para um vetor
    correl_array  = np.array(correl_data)

    # Criando um objeto de figura
    fig = go.Figure()

    # Criando o plot
    trace = go.Heatmap(z = correl_array,
                    x = colunas_correl,
                    y = colunas_correl,
                    colorscale = 'rdylgn',
                    )

    # Adicionando
    fig.add_trace(trace)

    # Atualizando o layout
    fig.update_layout(width = width, height = height, title = "Matriz de Correlação", title_x = 0.5)
    fig.update_traces(opacity = 0.75)
    fig.show()
    #py.plot(fig, filename="Matriz de Correlação_1", auto_open = False)

# Realizando a plotagem
matriz_correlacao(dataset_dummy_norm)

In [ ]:
# Obtendo as correlações para ativos e inativos
correl_data_ativos = dataset_dummy_norm.loc[dataset_dummy_norm['Attrition'] == 0].iloc[:, 1:-1].corr()
correl_data_inativos = dataset_dummy_norm.loc[dataset_dummy_norm['Attrition'] == 1].iloc[:, 1:-1].corr()

# Obtendo as colunas
colunas_correl_ativos = correl_data_ativos.columns.tolist()
colunas_correl_inativos = correl_data_inativos.columns.tolist()

# Convertendo para um vetor
correl_array_ativos  = np.array(correl_data_ativos)
correl_array_inativos  = np.array(correl_data_inativos)

# Criando um objeto de figura
fig = make_subplots(rows = 1, cols = 2, subplot_titles = ('Matriz de Correlações - Ativos', 'Matriz de Correlações - Inativos'))

# Criando os plots
trace_1 = go.Heatmap(z = correl_array_ativos,
                   x = colunas_correl_ativos,
                   y = colunas_correl_ativos,
                   colorscale = 'rdylgn',
                   showscale=False
                  )

trace_2 = go.Heatmap(z = correl_array_inativos,
                   x = colunas_correl_inativos,
                   y = colunas_correl_inativos,
                   colorscale = 'rdylgn',
                  )

# Adicionando
fig.append_trace(trace_1, row = 1, col = 1)
fig.append_trace(trace_2, row = 1, col = 2)

# Atualizando o layout
fig.update_layout(width = 1000, height = 500, title = "Matriz de Correlação - Comparativo", title_x = 0.5)
fig.update_yaxes(visible = False)
fig.update_xaxes(visible = False)
fig.update_traces(opacity = 0.75)
fig.show()

#### 7.2) Remoção de variáveis com alta correlação

In [ ]:
# Definindo um limite máximo de correlação permitido
limite = 0.8

# Obtendo os valores absolutos de correlação
matriz_correl_abs = dataset_label_encoder.corr().abs()
matriz_correl_abs.head()

# Removendo os valores abaixo da diagonal principal da matriz (incluindo a diagonal principal)
matriz_correl_abs_filtrada = matriz_correl_abs.where(np.triu(np.ones(matriz_correl_abs.shape), k=1).astype(np.bool))
matriz_correl_abs_filtrada.head()

# Identificando as colunas a serem removidas
features_colineares_label = [feature for feature in matriz_correl_abs_filtrada.columns if any(matriz_correl_abs_filtrada[feature] > limite)]

In [ ]:
# Definindo um limite máximo de correlação permitido
limite = 0.8

# Obtendo os valores absolutos de correlação
matriz_correl_abs = dataset_dummy_norm.corr().abs()
matriz_correl_abs.head()

# Removendo os valores abaixo da diagonal principal da matriz (incluindo a diagonal principal)
matriz_correl_abs_filtrada = matriz_correl_abs.where(np.triu(np.ones(matriz_correl_abs.shape), k=1).astype(np.bool))
matriz_correl_abs_filtrada.head()

# Identificando as colunas a serem removidas
features_colineares_dummy = [feature for feature in matriz_correl_abs_filtrada.columns if any(matriz_correl_abs_filtrada[feature] > limite)]

In [ ]:
# Observando as features colineares
display(features_colineares_label)
display(features_colineares_dummy)

['MonthlyIncome']

['Department_Sales', 'JobRole_Human Resources', 'JobRole_Sales Executive']

In [ ]:
# Removendo a coluna
dataset_label_encoder = dataset_label_encoder.drop(features_colineares_label, axis = 1)
dataset_label_encoder_norm = dataset_label_encoder_norm.drop(features_colineares_label, axis = 1)
dataset_dummy = dataset_dummy.drop(features_colineares_dummy, axis = 1)
dataset_dummy_norm = dataset_dummy_norm.drop(features_colineares_dummy, axis = 1)

### 8) Testes Estatísticos

Nesta sessão iremos checar a distribuição de algumas variáveis, assim como levantar hipóteses e realizar testes estatísticos para identificar diferenças entre amostras diferentes.

#### 8.1) Histogramas

In [ ]:
# Criando uma função para plotagem dos histogramas
def plot_histogramas(variaveis = "Lista de features a serem analisadas", dataset = dataset, width = 1500, height = 1800) -> object:
    # Definindo um limite superior para a plotagem
    limite_superior = int(len(variaveis)/2) if len(variaveis)/2 == len(variaveis)//2 else len(variaveis)//2 + 1
    
    if limite_superior > 1:
        cols = 2
    else:
        cols = 1

    # Plotando todos os gráficos
    fig = make_subplots(rows = limite_superior, cols = cols, subplot_titles = variaveis)
    contador = 0

    for i in range(1, limite_superior+1):
        for j in range(1, cols+1):
            try:
                # Criando o gráfico
                fig.add_trace(go.Histogram(x = dataset[variaveis[contador]], 
                                        marker_color = '#19d3f3', 
                                        showlegend = False, 
                                        opacity = 0.8), 
                            row = i, col = j)
                contador += 1
            except:
                pass

    fig.update_layout(width = width, height = height, title = "Distribuição das variáveis numéricas de interesse", plot_bgcolor = "white", title_x = 0.5)
    fig.update_yaxes(showline = True, linewidth = 1, linecolor = 'black', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE', rangemode = 'tozero')
    fig.update_xaxes(showline = True, linewidth = 1, linecolor = 'black', rangemode = 'nonnegative')
    fig.show()

# Realizando a plotagem
plot_histogramas(colunas_dist_plot_num)

#### 8.2) Testes de normalidade

In [ ]:
def qq_plot(variaveis = "Lista de features a serem analisadas", dataset = dataset, width = 1800, height = 2400) -> object:
    # Definindo um limite superior para a plotagem
    limite_superior = int(len(variaveis)/3) if len(variaveis)/3 == len(variaveis)//3 else len(variaveis)//3 + 1

    if limite_superior > 2:
        cols = 3
    elif limite_superior == 2:
        cols = 2
    else:
        cols = 1

    # Plotando todos os gráficos
    fig = make_subplots(rows = limite_superior, cols = cols, subplot_titles = variaveis, vertical_spacing = 0.05)
    contador = 0

    for i in range(1, limite_superior+1):
        for j in range(1, cols+1):
            try:
                # Legenda
                legenda = True if contador == 0 else False

                # Criando os datasets auxiliares
                qqplot_data = qqplot(dataset[variaveis[contador]], line='s').gca().lines
                plt.close()

                # Criando o gráfico
                fig.add_trace(go.Scatter(x = qqplot_data[0].get_xdata(),
                                        y = qqplot_data[0].get_ydata(),
                                        showlegend = False,
                                        mode = 'markers',
                                        marker_color = '#19d3f3'), 
                            row = i, col = j)
                fig.add_trace(go.Scatter(x = qqplot_data[1].get_xdata(),
                                        y = qqplot_data[1].get_ydata(),
                                        showlegend = False,
                                        mode = 'lines',
                                        line_color = '#636efa'),
                            row = i, col = j)
                contador += 1
            except:
                pass

    fig.update_layout(width = width, height = height, title = "Q-Q Plot para as variáveis contínuas", plot_bgcolor = "white", title_x = 0.5)
    fig.update_yaxes(showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE', zeroline = True, zerolinewidth = 1, zerolinecolor = '#EEEEEE')
    fig.update_xaxes(showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE', zeroline = True, zerolinewidth = 1, zerolinecolor = '#EEEEEE')
    fig.show()

# Realizando a plotagem
qq_plot(colunas_dist_plot_num)

In [ ]:
#for i in colunas_dist_plot_num:
#    qq_plot([i], width = 800, height = 500)

#### 8.3) Testes de variância

In [ ]:
# Usando o teste t de Student com a coluna Age
stats.ttest_ind(dataset.loc[dataset['Attrition'] == "Yes"]['Age'], dataset.loc[dataset['Attrition'] == "No"]['Age'])

Ttest_indResult(statistic=-6.1786638353072165, pvalue=8.356308021103649e-10)

#### 8.4) Boxplots

In [ ]:
fig = go.Figure()

# Adicionando os gráficos
fig.add_trace(go.Box(x = dataset.loc[dataset['Attrition'] == "Yes"]["Age"], name = "Inativos", marker_color = '#FFA07A', line_color = '#FFA07A'))
fig.add_trace(go.Box(x = dataset.loc[dataset['Attrition'] == "No"]["Age"], name = "Ativos", marker_color = '#90EE90', line_color = '#90EE90'))

# Atualizando o layout geral
fig.update_layout(barmode = 'overlay', width = 800, height = 500, showlegend = True, title = "Distribuição de Age", title_x = 0.5, plot_bgcolor = "white")

fig.show()
#py.plot(fig, filename="Boxplot de Age", auto_open = False)

### 9) Análise de Sobrevivência

In [ ]:
# Criando a função de plotagem das curvas de sobrevivência
def survival_curve(variavel = "Feature a ser analisada", limite = "Limiar para gerar a análise", dataset = dataset, width = 800, height = 500) -> object:
    # Criando um dataframe só com as variáveis de interesse
    variaveis = ['Attrition', 'YearsAtCompany']
    variaveis.append(variavel)
    dataset_aux = dataset[variaveis]
    dataset_aux['Attrition'] = dataset_aux['Attrition'].apply(lambda x: "Inativo" if x == 'Yes' else "Ativo")

    # Definindo um limite
    limite = limite

    # Transformando o tempo de casa em inteiro
    dataset_aux['YearsAtCompany'] = dataset_aux['YearsAtCompany'].apply(lambda x: int(x))

    # Categorizando a aba de pay position
    dataset_aux['Threshold 1'] = dataset_aux[variavel].apply(lambda x: 1 if x <= limite else 0)
    dataset_aux['Threshold 2'] = dataset_aux[variavel].apply(lambda x: 1 if x > limite else 0)

    # Criando a ocorrência
    dataset_aux['Inativo Threshold 1'] = dataset_aux.apply(lambda x: 1 if x['Threshold 1'] == 1 and x['Attrition'] == "Inativo" else 0, axis = 1)
    dataset_aux['Inativo Threshold 2'] = dataset_aux.apply(lambda x: 1 if x['Threshold 2'] == 1 and x['Attrition'] == "Inativo" else 0, axis = 1)
        
    # Removendo a coluna de Pay Position
    dataset_aux = dataset_aux.drop([variavel], axis = 1)

    # Criando as funções de sobrevivência para ativos
    # Até 1o limite
    kmf_1 = KaplanMeierFitter() 
    kmf_1.fit(dataset_aux['YearsAtCompany'], dataset_aux['Inativo Threshold 1'])

    # Acima do limite
    kmf_2 = KaplanMeierFitter() 
    kmf_2.fit(dataset_aux['YearsAtCompany'], dataset_aux['Inativo Threshold 2'])

    fig = go.Figure()

    fig.add_trace(go.Scatter(x = list(kmf_1.timeline), y = kmf_1.survival_function_['KM_estimate'],
                        mode = 'lines',
                        name = f'{variavel} até {limite}',
                        line = dict(color = 'rgba(75, 0, 130, 1)')))
    fig.add_trace(go.Scatter(x = list(kmf_1.timeline), y = kmf_1.confidence_interval_['KM_estimate_lower_0.95'],
                        mode = 'lines',
                        showlegend = False, 
                        line_color = 'rgba(0,0,0,0)'))
    fig.add_trace(go.Scatter(x = list(kmf_1.timeline), y = kmf_1.confidence_interval_['KM_estimate_upper_0.95'],
                        mode = 'lines',
                        line_color = 'rgba(0,0,0,0)',
                        fill = 'tonexty',
                        showlegend = False,
                        fillcolor = 'rgba(75, 0, 130, 0.2)'))
    fig.add_trace(go.Scatter(x = list(kmf_2.timeline), y = kmf_2.survival_function_['KM_estimate'],
                        mode = 'lines',
                        name = f'{variavel} acima de {limite}',
                        line = dict(color = 'rgba(0, 128, 128, 1)')))
    fig.add_trace(go.Scatter(x = list(kmf_2.timeline), y = kmf_2.confidence_interval_['KM_estimate_lower_0.95'],
                        mode = 'lines',
                        showlegend = False, 
                        line_color = 'rgba(0,0,0,0)'))
    fig.add_trace(go.Scatter(x = list(kmf_2.timeline), y = kmf_2.confidence_interval_['KM_estimate_upper_0.95'],
                        mode = 'lines',
                        line_color = 'rgba(0,0,0,0)',
                        fill = 'tonexty',
                        showlegend = False, 
                        fillcolor = 'rgba(0, 128, 128, 0.2)'))

    # Atualizando os títulos das abscissas 
    fig.update_xaxes(title_text = "Tempo de casa (em anos)", showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE', showline = True, linewidth = 1, linecolor = 'black')

    # Atualizando os títulos das coordenadas
    fig.update_yaxes(title_text = "Probabilidade de ser ativo", showline = True, linewidth = 1, linecolor = 'black', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE')

    # Atualizando o layout
    fig.update_layout(width = width, height = height, showlegend = True, title = f"Análise de Sobrevivência - {variavel}", legend = dict(yanchor = "top", y = 0.25, xanchor = "left", x = 0.05), plot_bgcolor = 'white', title_x = 0.5)

    fig.show()
    py.plot(fig, filename=f"Análise de Sobrevivencia - {variavel}", auto_open = False)

In [ ]:
# Plotando algumas curvas
survival_curve('DistanceFromHome', 2)
survival_curve('TrainingTimesLastYear', 3)

## Parte 3 - Análise Avançada

Modelos preditivos, feature importance, oversampling e escolha de um modelo

### 10) Tratamentos Adicionais

#### 10.1) Preparação da base

In [ ]:
# Definindo o dataset a ser utilizado
dataset_modelo = dataset_dummy_norm

In [ ]:
# Separando os datasets em classes (o que vamos prever) e preditores (as características que vamos analisar)
# Criação das classes
classes = dataset_modelo.iloc[:, 0].values

# Criação dos preditores
preditores = dataset_modelo.iloc[:, 1:-1].values

In [ ]:
# Divisão da base de dados entre treinamento e teste. Usamos 30% para testar e 70% para treinar. Random_state = 0 para sempre obter a mesma divisão da base quando o código for executado
x_train, x_test, y_train, y_test = train_test_split(preditores, classes, test_size = 0.2, random_state = 0)

#### 10.2) Criando uma base adicional com oversampling

In [ ]:
# Utilizando a técnica de oversampling
ros = RandomOverSampler(random_state = 0)
x_train_ros, y_train_ros = ros.fit_resample(x_train, y_train)

### 11) Definindo Funções de Avaliação

#### 11.1) Métricas de sucesso

In [ ]:
# Definição da função de plotagem das matrizes de confusão
def matriz_confusao(modelos = "Lista de modelos a serem comparados", x_test = x_test, y_test = y_test, width = 1000, height = 1000) -> object:
    # Definindo um limite superior para a plotagem
    limite_superior = int(len(modelos)/2) if len(modelos)/2 == len(modelos)//2 else len(modelos)//2 + 1

    if limite_superior > 1:
        cols = 2
    else:
        cols = 1

    # Criando os nomes
    nomes = [type(i).__name__ for i in modelos]

    # Plotando todos os gráficos
    fig = make_subplots(rows = limite_superior, cols = cols, subplot_titles = nomes, horizontal_spacing = 0.25, vertical_spacing = 0.2)
    contador = 0

    for i in range(1, limite_superior+1):
        for j in range(1, cols+1):
            try:
                # Obtendo a lista de predições
                modelo = modelos[contador]
                y_pred = modelo.predict(x_test)

                # Obtendo a matriz de confusão
                matriz_confusao = confusion_matrix(y_test, y_pred)

                # Obtendo as métricas
                tp = matriz_confusao[1, 1]  # Verdadeiro positivo
                fn = matriz_confusao[1, 0]  # Falso negativo
                fp = matriz_confusao[0, 1]  # Falso positivo
                tn = matriz_confusao[0, 0]  # Verdadeiro negativo

                # Matriz em termos de porcentagens
                matriz_porcentagem = [[round(fn/(tn+fn), 2), round(tp/(tp + fp), 2)], [round(tn/(tn+fn), 2), round(fp/(tp + fp), 2)]]

                # Criando o plot da matriz de confusão
                fig.append_trace(go.Heatmap(z = matriz_porcentagem, 
                                    x = ["Ativo (previsto)", "Inativo (previsto)"], 
                                    y = ["Inativo (real)", "Ativo (real)"],
                                    text = matriz_porcentagem,
                                    texttemplate = "%{text}",
                                    textfont = {"size": 16},
                                    xgap = 2, 
                                    ygap = 2, 
                                    colorscale = 'Blues', 
                                    showscale  = False),
                                    row = i,
                                    col = j)
                
                contador += 1
            except:
                pass

    fig["layout"].update(title = "Matrizes de confusão", title_x = 0.5, height = height, width = width, showlegend = False, plot_bgcolor = 'white', paper_bgcolor = 'white')
    fig.show()

In [ ]:
# Definição da função de plotagem das métricas de sucesso
def metricas_sucesso(modelos = "Lista de modelos a serem comparados", x_test = x_test, y_test = y_test, width = 1000, height = 1000) -> object:
    # Definindo um limite superior para a plotagem
    limite_superior = int(len(modelos)/2) if len(modelos)/2 == len(modelos)//2 else len(modelos)//2 + 1

    if limite_superior > 1:
        cols = 2
    else:
        cols = 1

    # Criando os nomes
    nomes = [type(i).__name__ for i in modelos]

    # Plotando todos os gráficos
    fig = make_subplots(rows = limite_superior, cols = cols, subplot_titles = nomes, horizontal_spacing = 0.25, vertical_spacing = 0.2)
    contador = 0

    for i in range(1, limite_superior+1):
        for j in range(1, cols+1):
            try:
                # Obtendo a lista de predições
                modelo = modelos[contador]
                y_pred = modelo.predict(x_test)

                # Obtendo a matriz de confusão
                matriz_confusao = confusion_matrix(y_test, y_pred)

                # Obtendo as métricas
                tp = matriz_confusao[1, 1]  # Verdadeiro positivo
                fn = matriz_confusao[1, 0]  # Falso negativo
                fp = matriz_confusao[0, 1]  # Falso positivo
                tn = matriz_confusao[0, 0]  # Verdadeiro negativo

                # Calculando as métricas de sucesso
                acuracia = round(((tp+tn)/(tp+tn+fp+fn)), 3)
                precisao = round((tp/(tp+fp)), 3)
                revocacao = round((tp/(tp+fn)), 3)
                f1_score = round((2*(((tp/(tp+fp))*(tp/(tp+fn)))/((tp/(tp+fp))+(tp/(tp+fn))))), 3)

                # Criando um dataset com as métricas de sucesso
                metricas_sucesso = pd.DataFrame(data = [[acuracia, precisao, revocacao, f1_score]])
                metricas_sucesso = metricas_sucesso.T

                colors = ['LightGoldenRodYellow', 'LightGreen', 'LightSalmon', 'LightSkyBlue']

                fig.append_trace(go.Bar(x = (metricas_sucesso[0].values), 
                                y = ['Accuracy', 'Precision', 'Recall', 'F1_Score'], 
                                text = metricas_sucesso[0].values,
                                textposition = 'auto',
                                orientation = 'h', 
                                opacity = 0.8,
                                marker = dict(
                                    color = colors,
                                    line = dict(
                                        color = '#000000',
                                        width = 1.5))),
                                 row = i,
                                 col = j)
                
                contador += 1
            except:
                pass

    fig.update_layout(title = "Métricas de Sucesso", title_x = 0.5, height = height, width = width, showlegend = False, plot_bgcolor = 'white', paper_bgcolor = 'white')
    fig.update_yaxes(showline = True, linewidth = 1, linecolor = 'black')
    fig.update_xaxes(showline = False, showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE', range = [0, 1.05])
    fig.show()

In [ ]:
# Definição da função de plotagem das métricas de sucesso
def roc_auc(modelos = "Lista de modelos a serem comparados", x_test = x_test, y_test = y_test, width = 1000, height = 500) -> object:
    # Definindo as cores
    colors = ['#8A2BE2', '#FF7F50', '#483D8B', '#1E90FF', '#FFD700', '#20B2AA', '#87CEFA']

    # Plotando todos os gráficos
    fig = go.Figure()

    for i in modelos:
        try:
            # Obtendo a lista de predições
            modelo = i
            y_pred = modelo.predict(x_test)

            # Calculando a curva roc
            roc_auc = round(roc_auc_score(y_test, modelo.predict_proba(x_test)[:, 1]) , 3)
            fpr, tpr, t = roc_curve(y_test, modelo.predict_proba(x_test)[:, 1])
            
            color = rd.choice(colors)
            colors.remove(color)
            
            fig.add_trace(go.Scatter(x = fpr,
                                y = tpr,
                                name = str(i),
                                opacity = 0.6,
                                line = dict(
                                    color = color,
                                    width = 2), 
                                fill = 'tozeroy'))
            
        except:
            pass

    fig.add_trace(go.Scatter(x = [0, 1],
                            y = [0, 1],
                            name = "Baseline",
                            line = dict(
                                color = ('black'),
                                width = 1.5,
                                dash = 'dot')))

    fig.update_layout(title = "Curvas ROC", title_x = 0.5, height = height, width = width, showlegend = True, plot_bgcolor = 'white', paper_bgcolor = 'white')
    fig.update_yaxes(showline = True, linewidth = 1, linecolor = 'black', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE', rangemode = 'tozero')
    fig.update_xaxes(showline = True, linewidth = 1, linecolor = 'black', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE', rangemode = 'nonnegative')
    fig.show()

#### 11.2) Avaliação geral do modelo

In [ ]:
# Definindo a função de plotagem das avaliações do modelo
def avaliacao_modelo(modelo = "Modelo preditivo a ser avaliado", x_test = x_test, y_test = y_test) -> object:
    # Obtendo a lista de predições
    y_pred = modelo.predict(x_test)

    # Obtendo a matriz de confusão
    matriz_confusao = confusion_matrix(y_test, y_pred)

    # Obtendo as métricas
    tp = matriz_confusao[1, 1]  # Verdadeiro positivo
    fn = matriz_confusao[1, 0]  # Falso negativo
    fp = matriz_confusao[0, 1]  # Falso positivo
    tn = matriz_confusao[0, 0]  # Verdadeiro negativo

    # Matriz em termos de porcentagens
    matriz_porcentagem = [[round(fn/(tn+fn), 2), round(tp/(tp + fp), 2)], [round(tn/(tn+fn), 2), round(fp/(tp + fp), 2)]]

    # Criando o plot da matriz de confusão
    trace_1 = go.Heatmap(z = matriz_porcentagem, 
                         x = ["Ativo (previsto)", "Inativo (previsto)"], 
                         y = ["Inativo (real)", "Ativo (real)"],
                         text = matriz_porcentagem,
                         texttemplate = "%{text}",
                         textfont = {"size": 16},
                         xgap = 2, 
                         ygap = 2, 
                         colorscale = 'Blues', 
                         showscale  = False)
    
    # Calculando as métricas de sucesso
    acuracia = round(((tp+tn)/(tp+tn+fp+fn)), 3)
    precisao = round((tp/(tp+fp)), 3)
    revocacao = round((tp/(tp+fn)), 3)
    f1_score = round((2*(((tp/(tp+fp))*(tp/(tp+fn)))/((tp/(tp+fp))+(tp/(tp+fn))))), 3)

    # Criando um dataset com as métricas de sucesso
    metricas_sucesso = pd.DataFrame(data = [[acuracia, precisao, revocacao, f1_score]])
    metricas_sucesso = metricas_sucesso.T

    colors = ['LightGoldenRodYellow', 'LightGreen', 'LightSalmon', 'LightSkyBlue']

    trace_2 = go.Bar(x = (metricas_sucesso[0].values), 
                    y = ['Acurácia', 'Precisão', 'Revocação', 'F1_Score'], 
                    text = metricas_sucesso[0].values,
                    textposition = 'auto',
                    orientation = 'h', 
                    opacity = 0.8,
                    marker = dict(
                        color = colors,
                        line = dict(
                            color = '#000000',
                            width = 1.5)))
    
    # Calculando a curva roc
    roc_auc = round(roc_auc_score(y_test, modelo.predict_proba(x_test)[:, 1]) , 3)
    fpr, tpr, t = roc_curve(y_test, modelo.predict_proba(x_test)[:, 1])

    trace_3 = go.Scatter(x = fpr,
                         y = tpr,
                         line = dict(
                             color = '#9370DB',
                             width = 2), 
                         fill = 'tozeroy')
    trace_4 = go.Scatter(x = [0, 1],
                         y = [0, 1],
                         line = dict(
                             color = ('black'),
                             width = 1.5,
                             dash = 'dot'))
    
    # Curva de precisão-revocação
    precision, recall, thresholds = precision_recall_curve(y_test, modelo.predict_proba(x_test)[:, 1])

    trace_5 = go.Scatter(x = recall, 
                         y = precision,
                         line = dict(
                             color = '#FF6347',
                             width = 2), 
                         fill = 'tozeroy')



    # Realizando a plotagem
    fig = tls.make_subplots(rows = 2, cols = 2, subplot_titles = ('Matriz de Confusão', 'Métricas de Sucesso', f'Curva ROC ({roc_auc})', 'Curva Precisão-Revocação'), vertical_spacing = 0.2)

    fig.append_trace(trace_1, row = 1, col = 1)
    fig.append_trace(trace_2, row = 1, col = 2)
    fig.append_trace(trace_3, row = 2, col = 1)
    fig.append_trace(trace_4, row = 2, col = 1)
    fig.append_trace(trace_5, row = 2, col = 2)

    fig["layout"].update(title = f'Performance do modelo: <b>{type(modelo).__name__}</b><br>', title_x = 0.5, height = 800, width = 1000, showlegend = False, plot_bgcolor = 'white', paper_bgcolor = 'white')
    fig["layout"]["xaxis2"].update((dict(range = [0, 1], showline = True, linewidth = 1, linecolor = 'black', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE')))
    fig["layout"]["yaxis2"].update((dict(showline = True, linewidth = 1, linecolor = 'black')))
    fig["layout"]["xaxis3"].update(dict(title = "Taxa de falsos positivos", showline = True, linewidth = 1, linecolor = 'black', rangemode = 'nonnegative', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE'))
    fig["layout"]["yaxis3"].update(dict(title = "Taxa de verdadeiros positivos", showline = True, linewidth = 1, linecolor = 'black', rangemode = 'tozero', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE'))
    fig["layout"]["xaxis4"].update(dict(title = "Revocação", range = [0, 1.05], showline = True, linewidth = 1, linecolor = 'black', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE'))
    fig["layout"]["yaxis4"].update(dict(title = "Precisão", range = [0, 1.05], showline = True, linewidth = 1, linecolor = 'black', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE'))

    fig.show()

#### 11.3) Feature importance

In [ ]:
def features_importances(modelo = "Modelo a ser analisado", width = 1000, height = 1000) -> object: 
    # Criando um dataframe com os valores
    coeficientes = pd.DataFrame(modelo.feature_importances_)
    atributos = pd.DataFrame(list(dataset_modelo))
    feature_importance = (pd.merge(coeficientes, atributos, left_index = True, right_index = True, how = "left"))
    feature_importance.columns = ["Coeficientes", "Features"]
    feature_importance = feature_importance.sort_values(by = "Coeficientes", ascending = True)
    feature_importance = feature_importance[feature_importance["Coeficientes"] !=0]
    
    # Criando o plot
    trace = go.Bar(x = feature_importance["Coeficientes"], 
                   y = feature_importance["Features"],
                   orientation = 'h',
                   marker = dict(
                       color = feature_importance["Coeficientes"],
                       colorscale = "OrRd",
                       line = dict(
                           width = 0.6, 
                           color = "black")))
    
    # Plotando
    fig = go.Figure()
    fig.add_trace(trace)
    fig.update_layout(width = width, height = height, title = f"Feature Importance - {type(modelo).__name__}", plot_bgcolor = "white", title_x = 0.5)
    fig.update_yaxes(showline = True, linewidth = 1, linecolor = 'black', rangemode = 'tozero')
    fig.update_xaxes(showline = True, linewidth = 1, linecolor = 'black', showgrid = True, gridwidth = 1, gridcolor = '#EEEEEE', rangemode = 'nonnegative')
    fig.show()

### 12) Criando Modelos Preditivos

#### 12.1) Regressão Logística

In [ ]:
# Criando o modelo
baseline_logisticregression = LogisticRegression()
baseline_logisticregression.fit(x_train, y_train)

LogisticRegression()

In [ ]:
# Avaliando o modelo
avaliacao_modelo(baseline_logisticregression)

/usr/local/lib/python3.7/dist-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



#### 12.2) Árvore de Decisão

In [ ]:
# Criando o modelo
baseline_decisiontree = DecisionTreeClassifier()
baseline_decisiontree.fit(x_train, y_train)

DecisionTreeClassifier()

In [ ]:
# Avaliando o modelo
avaliacao_modelo(baseline_decisiontree)

/usr/local/lib/python3.7/dist-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



#### 12.3) Árvores de Decisão - Florestas (Primeiro algoritimo) 

In [ ]:
# Criando o modelo
baseline_randomforrest = RandomForestClassifier()
baseline_randomforrest.fit(x_train, y_train)

RandomForestClassifier()

In [ ]:
# Avaliando o modelo
avaliacao_modelo(baseline_randomforrest)

/usr/local/lib/python3.7/dist-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



#### 12.4) Árvores de Decisão - Florestas (Segundo algoritmo)

In [ ]:
# Criando o modelo
baseline_extratrees = ExtraTreesClassifier()
baseline_extratrees.fit(x_train, y_train)

ExtraTreesClassifier()

In [ ]:
# Avaliando o modelo
avaliacao_modelo(baseline_extratrees)

/usr/local/lib/python3.7/dist-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



#### 12.5) K-nearest Neighbors 

In [ ]:
# Criando o modelo
baseline_knn = KNeighborsClassifier()
baseline_knn.fit(x_train, y_train)

KNeighborsClassifier()

In [ ]:
# Avaliando o modelo
avaliacao_modelo(baseline_knn)

/usr/local/lib/python3.7/dist-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



#### 12.6) Naïve Bayes

In [ ]:
# Criando o modelo
baseline_naivebayes = GaussianNB()
baseline_naivebayes.fit(x_train, y_train)

GaussianNB()

In [ ]:
# Avaliando o modelo
avaliacao_modelo(baseline_naivebayes)

/usr/local/lib/python3.7/dist-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



#### 12.7) Máquina de Vetor de Suporte (SVM)

In [ ]:
# Criando o modelo
baseline_svm = SVC(probability = True)
baseline_svm.fit(x_train, y_train)

SVC(probability=True)

In [ ]:
# Avaliando o modelo
avaliacao_modelo(baseline_svm)

/usr/local/lib/python3.7/dist-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



#### 12.8) Gradient Boosting (XGBoost)

In [ ]:
# Criando o modelo
baseline_xgboost = xgb.XGBClassifier()
baseline_xgboost.fit(x_train, y_train)

XGBClassifier()

In [ ]:
# Avaliando o modelo
avaliacao_modelo(baseline_xgboost)

/usr/local/lib/python3.7/dist-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



#### 12.9) Redes Neurais (Multilayer Perceptron)

In [ ]:
# Criando o modelo
baseline_mlp = MLPClassifier()
baseline_mlp.fit(x_train, y_train)

MLPClassifier()

In [ ]:
# Avaliando o modelo
avaliacao_modelo(baseline_mlp)

/usr/local/lib/python3.7/dist-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



### 13) Comparando os Modelos Escolhidos

#### 13.1) Métricas de sucesso

In [ ]:
# Função para comparar as métricas de sucesso
metricas_sucesso([baseline_logisticregression, baseline_extratrees, baseline_naivebayes, baseline_mlp])

#### 13.2) Matrizes de confusão

In [ ]:
# Função para comparar as matrizes de confusão
matriz_confusao([baseline_logisticregression, baseline_extratrees, baseline_naivebayes, baseline_mlp])

#### 13.3) Curva ROC e AUC

In [ ]:
# Função para comparar as curvas ROC e AUC
roc_auc([baseline_logisticregression, baseline_extratrees, baseline_naivebayes, baseline_mlp])

### 14) Avaliando o Oversampling

#### 14.1) Criando os novos modelos

In [ ]:
# Criando o modelo de regressão logística
baseline_ros_logisticregression = LogisticRegression()
baseline_ros_logisticregression.fit(x_train_ros, y_train_ros)

LogisticRegression()

In [ ]:
# Criando o modelo de árvores de decisão (florestas)
baseline_ros_extratrees = ExtraTreesClassifier()
baseline_ros_extratrees.fit(x_train_ros, y_train_ros)

ExtraTreesClassifier()

In [ ]:
# Criando o modelo de naïve bayes
baseline_ros_naivebayes = GaussianNB()
baseline_ros_naivebayes.fit(x_train_ros, y_train_ros)

GaussianNB()

In [ ]:
# Criando o modelo de redes neurais
baseline_ros_mlp = MLPClassifier()
baseline_ros_mlp.fit(x_train_ros, y_train_ros)

MLPClassifier()

#### 14.2) Comparando os baselines

In [ ]:
# Função para comparar as métricas de sucesso
metricas_sucesso([baseline_logisticregression, baseline_extratrees, baseline_naivebayes, baseline_mlp])
print("")
metricas_sucesso([baseline_ros_logisticregression, baseline_ros_extratrees, baseline_ros_naivebayes, baseline_ros_mlp])

In [ ]:
# Função para comparar as matrizes de confusão
matriz_confusao([baseline_logisticregression, baseline_extratrees, baseline_naivebayes, baseline_mlp])
print("")
matriz_confusao([baseline_ros_logisticregression, baseline_ros_extratrees, baseline_ros_naivebayes, baseline_ros_mlp])

### 15) Feature Importance

In [ ]:
# Plotando as feature importances
features_importances(baseline_extratrees)
features_importances(baseline_randomforrest)
features_importances(baseline_xgboost)

## Parte 4 - Análise Expert

Feature engineering, feature selection, grid search, random search e avaliação das métricas de sucesso

### 16) Feature Engineering

#### 16.1) Combinando algumas features

In [ ]:
# Utilizando o dataset original para a construção de novas features
dataset_novo = dataset.copy()

In [ ]:
# Média de satisfação total
dataset_novo['New_TotalSatisfactionMean'] = (dataset_novo['RelationshipSatisfaction']  + 
                                          dataset_novo['EnvironmentSatisfaction'] + 
                                          dataset_novo['JobSatisfaction'] + 
                                          dataset_novo['JobInvolvement'] + 
                                          dataset_novo['WorkLifeBalance'])/5

# Média de satisfação com o ambiente
dataset_novo['New_RelSatisfMean'] = (dataset_novo['RelationshipSatisfaction']  + dataset_novo['EnvironmentSatisfaction'])/2

# Média de satisfação com o emprego
dataset_novo['New_JobSatisfMean'] = (dataset_novo['JobSatisfaction']  + dataset_novo['JobInvolvement'])/2

# Média de anos por empresa
dataset_novo['New_YearsPerCompanyMean'] = round(dataset_novo['TotalWorkingYears']/dataset_novo['NumCompaniesWorked'], 2)
dataset_novo['New_YearsPerCompanyMean'] = dataset_novo['New_YearsPerCompanyMean'].apply(lambda x: 0 if x == math.inf else x)

#### 16.2) Funções de criação das novas features

In [ ]:
# Funções de criação de novas features
def alta_mudanca_empresa(dataset) :
    if dataset['NumCompaniesWorked'] > 4:
        return 1
    else:
        return 0

def engajamento_job(dataset) :
    if dataset['JobInvolvement'] < 2.5 :
        return 1
    else:
        return 0

def baixa_satistacao(dataset) : 
    if  dataset['New_TotalSatisfactionMean'] < 2.35 :
        return 1
    else : 
        return 0

def novo_salario_baixo(dataset) : 
    if dataset['Age'] < 35 and dataset['Age'] > 23 and (dataset['MonthlyIncome'] < 3500):
        return 1
    else : 
        return 0

def area_sales(dataset) :
    if dataset['Department'] == 'Sales':
        return 1
    else:
        return 0

#### 16.3) Aplicando as funções de criação de novas features

In [ ]:
# Aplicando as funções
dataset_novo['New_MoovingPeople'] = dataset_novo.apply(lambda dataset_novo: alta_mudanca_empresa(dataset_novo), axis = 1)
dataset_novo['New_LowJobInvolvment'] = dataset_novo.apply(lambda dataset_novo: engajamento_job(dataset_novo), axis = 1)
dataset_novo['New_Unsatisfied'] = dataset_novo.apply(lambda dataset_novo: baixa_satistacao(dataset_novo), axis = 1)
dataset_novo['New_YoungBadPaid'] = dataset_novo.apply(lambda dataset_novo: novo_salario_baixo(dataset_novo), axis = 1)
dataset_novo['New_Sales'] = dataset_novo.apply(lambda dataset_novo: area_sales(dataset_novo), axis = 1)

### 17) Observando as Novas Variáveis

#### 17.1) Criando um dataset auxiliar

In [ ]:
# Selecionando apenas features de interesse
dataset_novas_features = dataset_novo[['Attrition', 'New_TotalSatisfactionMean', 'New_RelSatisfMean', 'New_JobSatisfMean', 'New_YearsPerCompanyMean', 'New_MoovingPeople', 'New_LowJobInvolvment', 'New_Unsatisfied', 'New_YoungBadPaid', 'New_Sales']]

#### 17.2) Análise exploratória

In [ ]:
# Função de plotagem das distribuições
dist_plot(['New_TotalSatisfactionMean', 'New_RelSatisfMean', 'New_JobSatisfMean', 'New_YearsPerCompanyMean'], dataset = dataset_novas_features, width = 1200, height = 800)

In [ ]:
# # Função de plotagem das barras
bar_plot_num(['New_TotalSatisfactionMean', 'New_RelSatisfMean', 'New_JobSatisfMean'], dataset = dataset_novas_features, width = 1200, height = 800)

#### 17.3) Correlação

In [ ]:
# Tratando a coluna de 'Attrition'
dataset_novas_features_corr = dataset_novas_features.copy()
dataset_novas_features_corr['Attrition'] = dataset_novas_features_corr['Attrition'].apply(lambda x: 1 if x == "Yes" else 0)

In [ ]:
# Função de plotagem da matriz de correlação
matriz_correlacao(dataset_novas_features_corr)

#### 17.4) Normalidade

In [ ]:
# Função de plotagem dos histogramas
plot_histogramas(['New_TotalSatisfactionMean', 'New_RelSatisfMean', 'New_JobSatisfMean', 'New_YearsPerCompanyMean'], dataset = dataset_novas_features, width = 1200, height = 800)

In [ ]:
# Função de plotagem dos qq_plots
qq_plot(['New_TotalSatisfactionMean', 'New_RelSatisfMean', 'New_JobSatisfMean', 'New_YearsPerCompanyMean'], dataset = dataset_novas_features, width = 800, height = 900)

### 18) Nova Versão do Dataset

#### 18.1) Normalização

In [ ]:
# Separando em dois novos datasets
dataset_novas_features_raw = dataset_novas_features.drop(['Attrition'], axis = 1)

In [ ]:
# Deixando o dataset numa escala padronizada
dataset_norm_novas_features = min_max_scaler.fit_transform(dataset_novas_features_raw)  # utilizando o mesmo objeto de encoding
dataset_norm_novas_features = pd.DataFrame(dataset_norm_novas_features, columns = dataset_novas_features_raw.columns)

In [ ]:
# Unindo os datasets
dataset_dummy_norm_novo = pd.concat([dataset_dummy_norm, dataset_norm_novas_features], axis = 1)

In [ ]:
# Criando como versão final
dataset_final = dataset_dummy_norm_novo.copy()

#### 18.2) Preparação os dados para a feature selection

In [ ]:
# Definindo o dataset a ser utilizado
dataset_modelo_2 = dataset_final

In [ ]:
# Separando os datasets em classes (o que vamos prever) e preditores (as características que vamos analisar)
# Criação das classes
classes_2 = dataset_modelo_2.iloc[:, 0].values
classes_2_dataframe = dataset_modelo_2[['Attrition']]

# Criação dos preditores
preditores_2 = dataset_modelo_2.iloc[:, 1:-1].values
preditores_2_dataframe = dataset_modelo_2.iloc[:, 1:-1]

In [ ]:
# Divisão da base de dados entre treinamento e teste. Usamos 30% para testar e 70% para treinar. Random_state = 0 para sempre obter a mesma divisão da base quando o código for executado
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(preditores_2, classes_2, test_size = 0.2, random_state = 0)

In [ ]:
# Criando dataframes com os dados de treino e teste
x_train_2_dataframe = pd.DataFrame(x_train_2, columns = preditores_2_dataframe.columns)
x_test_2_dataframe = pd.DataFrame(x_test_2, columns = preditores_2_dataframe.columns)
y_train_2_dataframe = pd.DataFrame(y_train_2, columns = classes_2_dataframe.columns)
y_test_2_dataframe = pd.DataFrame(y_test_2, columns = classes_2_dataframe.columns)

### 19) Feature Selection

#### 19.1) Criando as funções de seleção

In [ ]:
# Correlação
def f_selection_correlacao(num_feats, x = x_train_2_dataframe, y = y_train_2_dataframe) -> object:
    cor_list = []
    feature_name = x.columns.tolist()
    # Calculando a correlação
    for i in x.columns.tolist():
        cor = np.corrcoef(x[i], y['Attrition'])[0, 1]
        cor_list.append(cor)
    # Substituindo os valores NaN
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # Nome das features
    cor_feature = x.iloc[:, np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # Retornando True se for uma feature selecionada
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

In [ ]:
# RFE (Recursive Feature Elimination)
def f_selection_rfe(num_feats, estimator, x = x_train_2, y = y_train_2, x_dataframe = x_train_2_dataframe) -> object:
    rfe_selector = RFE(estimator = estimator, n_features_to_select = num_feats, step = 2, verbose = 5)
    rfe_selector.fit(x, y)
    rfe_support = rfe_selector.get_support()
    rfe_feature = x_dataframe.loc[:, rfe_support].columns.tolist()
    return rfe_support, rfe_feature

In [ ]:
# A partir de um modelo
def f_selection_from_model(num_feats, estimator = LogisticRegression(), x = x_train_2, y = y_train_2, x_dataframe = x_train_2_dataframe) -> object:
    from_model_selector = SelectFromModel(estimator, max_features = num_feats)
    from_model_selector.fit(x, y)
    from_model_support = from_model_selector.get_support()
    from_model_feature = x_dataframe.loc[:, from_model_support].columns.tolist()
    return from_model_support, from_model_feature

#### 19.2) Criando um dataframe de feature selection

In [ ]:
# Definindo os nomes das features
features_total = x_train_2_dataframe.columns

# Definindo quantas features iremos utilizar
num_features = 30

In [ ]:
# Criando o dataframe
feature_selection_df = pd.DataFrame({'Feature': features_total, 
                                     'Correlacao': f_selection_correlacao(num_features)[0], 
                                     'RFE_RegressaoLogistica': f_selection_rfe(num_features, LogisticRegression())[0],
                                     'RFE_RandomForrest': f_selection_rfe(num_features, RandomForestClassifier(n_estimators = 1000, max_depth = 4, class_weight = 'balanced'))[0],
                                     'RegressaoLogistica': f_selection_from_model(num_features, LogisticRegression())[0],
                                     'RandomForest': f_selection_from_model(num_features, RandomForestClassifier(n_estimators = 1000, max_depth = 4, class_weight = 'balanced'))[0], 
                                     'ExtraTrees': f_selection_from_model(num_features, ExtraTreesClassifier(n_estimators = 1000, max_depth = 4, class_weight = 'balanced'))[0],
                                     'XGBoost': f_selection_from_model(num_features, xgb.XGBClassifier())[0]})

Fitting estimator with 86 features.
Fitting estimator with 84 features.
Fitting estimator with 82 features.
Fitting estimator with 80 features.
Fitting estimator with 78 features.
Fitting estimator with 76 features.
Fitting estimator with 74 features.
Fitting estimator with 72 features.
Fitting estimator with 70 features.
Fitting estimator with 68 features.
Fitting estimator with 66 features.
Fitting estimator with 64 features.
Fitting estimator with 62 features.
Fitting estimator with 60 features.
Fitting estimator with 58 features.
Fitting estimator with 56 features.
Fitting estimator with 54 features.
Fitting estimator with 52 features.
Fitting estimator with 50 features.
Fitting estimator with 48 features.
Fitting estimator with 46 features.
Fitting estimator with 44 features.
Fitting estimator with 42 features.
Fitting estimator with 40 features.
Fitting estimator with 38 features.
Fitting estimator with 36 features.
Fitting estimator with 34 features.
Fitting estimator with 32 fe

In [ ]:
# Contando o número de ocorrências
feature_selection_df['Total'] = np.sum(feature_selection_df, axis = 1)

# Ordenando o dataframe
feature_selection_df = feature_selection_df.sort_values(['Total', 'Feature'], ascending = False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(num_features)

,Feature,Correlacao,RFE_RegressaoLogistica,RFE_RandomForrest,RegressaoLogistica,RandomForest,ExtraTrees,XGBoost,Total
1,YearsWithCurrManager,True,True,True,True,True,True,True,7
2,YearsInCurrentRole,True,True,True,True,True,True,True,7
3,TotalWorkingYears,True,True,True,True,True,True,True,7
4,OverTime,True,True,True,True,True,True,True,7
5,New_TotalSatisfactionMean,True,True,True,True,True,True,True,7
6,MonthlyIncome,True,True,True,True,True,True,True,7
7,JobLevel_2,True,True,True,True,True,True,True,7
8,EnvironmentSatisfaction_1,True,True,True,True,True,True,True,7
9,Age,True,True,True,True,True,True,True,7
10,New_YearsPerCompanyMean,True,True,True,True,True,False,True,6


#### 19.3) Criando as versões finais dos dados para treino e teste

In [ ]:
# Definindo o dataset a ser utilizado
features_utilizadas = feature_selection_df.loc[feature_selection_df['Total'] > 3]['Feature']
dataset_modelo_final = dataset_final[features_utilizadas]
dataset_modelo_final.insert(0, 'Attrition', dataset_final['Attrition'])

In [ ]:
# Separando os datasets em classes (o que vamos prever) e preditores (as características que vamos analisar)
# Criação das classes
classes_final = dataset_modelo_final.iloc[:, 0].values

# Criação dos preditores
preditores_final = dataset_modelo_final.iloc[:, 1:-1].values

In [ ]:
# Divisão da base de dados entre treinamento e teste. Usamos 30% para testar e 70% para treinar. Random_state = 0 para sempre obter a mesma divisão da base quando o código for executado
x_train_final, x_test_final, y_train_final, y_test_final = train_test_split(preditores_final, classes_final, test_size = 0.2, random_state = 0)

In [ ]:
# Utilizando a técnica de oversampling
ros = RandomOverSampler(random_state = 42)
x_train_final_ros, y_train_final_ros = ros.fit_resample(x_train_final, y_train_final)

### 20) Avaliando os Modelos no Dataset Final

#### 20.1) Criação dos modelos

In [ ]:
# Sem hiperparâmetros
regressao_logistica = LogisticRegression()
regressao_logistica.fit(x_train_final, y_train_final)

extra_trees = ExtraTreesClassifier()
extra_trees.fit(x_train_final, y_train_final)

naive_bayes = GaussianNB()
naive_bayes.fit(x_train_final, y_train_final)

xgboost = xgb.XGBClassifier()
xgboost.fit(x_train_final, y_train_final)

XGBClassifier()

In [ ]:
# Com alguns hiperparâmetros
regressao_logistica_hiper = LogisticRegression()
regressao_logistica_hiper.fit(x_train_final, y_train_final)

extra_trees_hiper = ExtraTreesClassifier(n_estimators = 1000, max_depth = 4, class_weight = 'balanced')
extra_trees_hiper.fit(x_train_final, y_train_final)

naive_bayes_hiper = GaussianNB()
naive_bayes_hiper.fit(x_train_final, y_train_final)

xgboost_hiper = xgb.XGBClassifier()
xgboost_hiper.fit(x_train_final, y_train_final)

XGBClassifier()

In [ ]:
# Com alguns hiperparâmetros e oversampling
regressao_logistica_hiper_ros = LogisticRegression()
regressao_logistica_hiper_ros.fit(x_train_final_ros, y_train_final_ros)

extra_trees_hiper_ros = ExtraTreesClassifier(n_estimators = 1000, max_depth = 4, class_weight = 'balanced')
extra_trees_hiper_ros.fit(x_train_final_ros, y_train_final_ros)

naive_bayes_hiper_ros = GaussianNB()
naive_bayes_hiper_ros.fit(x_train_final_ros, y_train_final_ros)

xgboost_hiper_ros = xgb.XGBClassifier()
xgboost_hiper_ros.fit(x_train_final_ros, y_train_final_ros)

XGBClassifier()

#### 20.2) Avaliação dos modelos

In [ ]:
# Sem hiperparâmetros
metricas_sucesso([regressao_logistica, extra_trees, naive_bayes, xgboost], x_test = x_test_final, y_test = y_test_final)

In [ ]:
# Com alguns hiperparâmetros
metricas_sucesso([regressao_logistica_hiper, extra_trees_hiper, naive_bayes_hiper, xgboost_hiper], x_test = x_test_final, y_test = y_test_final)

In [ ]:
# Com alguns hiperparâmetros e oversampling
metricas_sucesso([regressao_logistica_hiper_ros, extra_trees_hiper_ros, naive_bayes_hiper_ros, xgboost_hiper_ros], x_test = x_test_final, y_test = y_test_final)

#### 20.3) Matrizes de confusão

In [ ]:
# Sem hiperparâmetros
matriz_confusao([regressao_logistica, extra_trees, naive_bayes, xgboost], x_test = x_test_final, y_test = y_test_final)

In [ ]:
# Com alguns hiperparâmetros
matriz_confusao([regressao_logistica_hiper, extra_trees_hiper, naive_bayes_hiper, xgboost_hiper], x_test = x_test_final, y_test = y_test_final)

In [ ]:
# Com alguns hiperparâmetros e oversampling
matriz_confusao([regressao_logistica_hiper_ros, extra_trees_hiper_ros, naive_bayes_hiper_ros, xgboost_hiper_ros], x_test = x_test_final, y_test = y_test_final)

#### 20.4) Comparando as curvas ROC e AUC

In [ ]:
# Com alguns hiperparâmetros
roc_auc([regressao_logistica_hiper, extra_trees_hiper, naive_bayes_hiper, xgboost_hiper], x_test = x_test_final, y_test = y_test_final)

### 21) Otimização dos Hiperparâmetros

#### 20.1) Definindo os hiperparâmetros a serem utilizados

In [ ]:
# Dicionário de parâmetros
hiperparametros = {
    'n_estimators': [100, 200, 400, 700, 1000, 1200, 1500, 2000],
    'max_depth': [3, 4, 5, 6, 7, 8, 10, 15],
    'min_samples_split': [2, 4, 6, 7, 8, 9, 10, 15, 20, 30, 50],
    'min_samples_leaf': [2, 4, 6,  7, 8, 9, 10, 15, 20, 30, 50],
    'class_weight': ('balanced',)
}

#### 20.2) Random Search

In [ ]:
# Definindo o modelo
model = ExtraTreesClassifier()

# Quantidade de testes
total_iteracoes = 300

# Realizando o Random Search
random_search = RandomizedSearchCV(model, param_distributions = hiperparametros, n_iter = total_iteracoes, scoring = 'recall', random_state = 10)
#random_search.fit(x_train_final, y_train_final)

# Observando os melhores parâmetros
#random_search.best_params_

#### 20.3) Grid Search

In [ ]:
# Definindo o modelo
model = ExtraTreesClassifier()

# Quantidade de testes
total_iteracoes = 50

# Realizando o Random Search
grid_search = GridSearchCV(model, param_grid = hiperparametros, scoring = 'recall')
#grid_search.fit(x_train_final, y_train_final)

# Observando os melhores parâmetros
#grid_search.best_params_

#### 20.4) Modelo com os melhores hiperparâmetros

In [ ]:
# Criando o modelo final
modelo_preditivo = ExtraTreesClassifier(n_estimators = 1200, max_depth = 8, min_samples_leaf = 50, min_samples_split = 9, class_weight = 'balanced')
modelo_preditivo.fit(x_train_final, y_train_final)

ExtraTreesClassifier(class_weight='balanced', max_depth=8, min_samples_leaf=50,
                     min_samples_split=9, n_estimators=1200)

In [ ]:
# Avaliando o modelo
avaliacao_modelo(modelo_preditivo, x_test = x_test_final, y_test = y_test_final)

/usr/local/lib/python3.7/dist-packages/plotly/tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead

